In [4]:
import tensorflow as tf
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline  

from datasets import flower_dataset
#from models import models
from backbone import inception
from preprocessing import inception_preprocessing

In [8]:
import math, os

In [5]:
dataset_dir = '/home/dan/prj/datasets/flowers/flower_example1/train'

In [2]:
from models import bregman

In [6]:
filepaths, class_name_to_ids = flower_dataset.load_categorical_data(dataset_dir)

In [10]:
categorical_batch_size = math.floor(64 / len(class_name_to_ids.keys()))

In [11]:
category_balanced_data = flower_dataset.category_balancing_input_fn(
                filepaths, class_name_to_ids, categorical_batch_size)